<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Image-Recongnition-Case-with-CNN-By-Tensorflow" data-toc-modified-id="Image-Recongnition-Case-with-CNN-By-Tensorflow-0.1">Image Recongnition Case with CNN By Tensorflow</a></span><ul class="toc-item"><li><span><a href="#1.Loading-data" data-toc-modified-id="1.Loading-data-0.1.1">1.Loading data</a></span><ul class="toc-item"><li><span><a href="#1.1创建TFReord文件(create-TFRecord-file)" data-toc-modified-id="1.1创建TFReord文件(create-TFRecord-file)-0.1.1.1">1.1创建TFReord文件(create TFRecord file)</a></span></li><li><span><a href="#1.2-读取TFRecord文件" data-toc-modified-id="1.2-读取TFRecord文件-0.1.1.2">1.2 读取TFRecord文件</a></span></li></ul></li><li><span><a href="#3.Alexnet-model" data-toc-modified-id="3.Alexnet-model-0.1.2">3.Alexnet model</a></span><ul class="toc-item"><li><span><a href="#3.1-define-network-layers" data-toc-modified-id="3.1-define-network-layers-0.1.2.1">3.1 define network layers</a></span></li><li><span><a href="#3.2-implementation-of-Alexnet-architecture" data-toc-modified-id="3.2-implementation-of-Alexnet-architecture-0.1.2.2">3.2 implementation of Alexnet architecture</a></span></li><li><span><a href="#3.3-finetune_alexnet" data-toc-modified-id="3.3-finetune_alexnet-0.1.2.3">3.3 finetune_alexnet</a></span></li></ul></li></ul></li></ul></li></ul></div>

## Image Recongnition Case with CNN By Tensorflow
Refrence List
- Coursera course [Deep Learning Specialization](https://www.coursera.org/specializations/deep-learning) by Andrew Ng
- [An Introduction to Implementing Neural Networks using Tensorflow](https://www.analyticsvidhya.com/blog/2016/10/an-introduction-to-implementing-neural-networks-using-tensorflow/)
- [Deep Learning with Tensorflow](https://github.com/caicloud/tensorflow-tutorial)

- The arthitechture of Inception-v3 is an classic [Convolutional Neural Network](https://en.wikipedia.org/wiki/Convolutional_neural_network)
 - stacking the output layers convoluting by different filter 
 - the different filter can detect various scale feature 
 - keeping scale invariance property
- Tensorflow is a low level library
- To define a neural network architecture, you should have fundamental knowledge of NN:
 - initialize parameter
 - forward propagation
 - compute cost
 - backward propagation
 - update parameter

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.python.framework import ops

from sklearn.model_selection import train_test_split
import os
path = r"F:\Diverse\statistics\Python_data_analysis\Kaggle项目\20170926_MNIST\dataset"
os.chdir(path)

%matplotlib inline
np.random.seed(1)

### 1.Loading data
- create a TFRecord file
- loading data API
 - tf.data.dataset
 - tf.data.TFRecordDataset

#### 1.1创建TFReord文件(create TFRecord file)

#### 1.2 读取TFRecord文件

In [2]:
#解析方法2
def parser(example):
    features={"image_raw": tf.FixedLenFeature([],tf.string),
          "labels": tf.FixedLenFeature([],tf.int64)}
    feature = tf.parse_single_example(example,features)
    images = tf.decode_raw(feature["image_raw"],tf.int64)
    label = tf.cast(feature["labels"], tf.int8)
    return images,label

def get_data(batch, epoch):
    filename = r"mnist.tfrecords" #地址必须已经存在
    dataset = tf.data.TFRecordDataset([filename])
    maps = dataset.map(parser)
    batch_data = maps.repeat(epoch)
    batch_data = batch_data.batch(batch) 
    iterator =batch_data.make_one_shot_iterator() #iterator对象
    one_element = iterator.get_next() #tensor对象
    return one_element

'''
sess = tf.Session()
for i in range(2):
    image_data,labels_data=sess.run(one_element)
'''

'\nsess = tf.Session()\nfor i in range(2):\n    image_data,labels_data=sess.run(one_element)\n'

In [5]:
test = pd.read_csv("test.csv")

To reduce training time, I just pick up 15% example for fitting my model.

In [6]:
train_sample = train.sample(frac=0.15)
X_train_flatten = train_sample.iloc[:,1:].values
Y_train_onehot = convert_to_one_hot(train_sample.iloc[:,0].values,10)
X_test_flatten = test.values

In [7]:
X_train, X_dev, Y_train, Y_dev = train_test_split(X_train_flatten, Y_train_onehot, random_state=42)

In [8]:
X_train = X_train/255
X_dev = X_dev/255
X_test = X_test_flatten/255

In [9]:
print ("number of training examples = " + str(X_train.shape[1]))
print ("number of dev examples = " + str(X_dev.shape[1]))
print ("number of test examples = " + str(X_test.shape[1]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_dev shape: " + str(X_dev.shape))
print ("Y_dev shape: " + str(Y_dev.shape))
print ("X_test shape: " + str(X_test.shape))

number of training examples = 784
number of dev examples = 784
number of test examples = 784
X_train shape: (4725, 784)
Y_train shape: (4725, 10)
X_dev shape: (1575, 784)
Y_dev shape: (1575, 10)
X_test shape: (28000, 784)


### 3.Alexnet model
- 3 type layer: convolution layer & local response normalization layer & fullly connected layer
- overall architecture: 5 convolution + 3 fullly connected layer
 - input layer
 - (conv + local response normalization + maxpooling) x 2
 - conv x 2
 - conv + maxpooling
 - (fullly-connected + dropout)x 2
 - output layer(the 3rd fc)

#### 3.1 define network layers

In [19]:
def conv(x, filter_size, num_filters, stride, name, padding="SAME"):
    # create parameters of filter
    filter_height, filter_weight = filter_size
    stride_x, stride_y = stride
    input_channels = x.get_shape()[-1]
    
    # create convolution
    with tf.variable_scope(name) as scope:
        weights = tf.get_variable("weights",shape=[filter_height,
                                                  filter_weight,
                                                  input_channels,
                                                  num_filters],
                                 initializer=tf.truncated_normal_initializer(stddev=0.1))
        biases = tf.get_variable("biases", shape=[num_filters],
                                initializer=tf.constant_initializer(0))
        conv = tf.nn.conv2d(x, weights, strides=[1, stride_x, stride_y, 1],
                            padding=padding)
        bias = tf.nn.bias_add(conv, biases)
        relu = tf.nn.relu(bias, name=scope.name)
    
    return relu

def fc(x, num_in, num_out, name, relu=True):
    #create a fully connected layer
    with tf.variable_scope(name) as scope:
        weights = tf.get_variable("weights",shape=[num_in,num_out],
                                  initializer=tf.truncated_normal_initializer(stddev=0.1))
        biases = tf.get_variable("biases", shape=[num_out],
                                initializer=tf.constant_initializer(0))
        act = tf.nn.xw_plus_b(x, weights, biases, name=scope.name)
        if relu:
            relu = tf.nn.relu(act)
            return relu
        else:
            return act                

def max_pool(x, filter_size, stride, name, padding="SAME"):
    filter_height, filter_weight = filter_size
    stride_x, stride_y = stride
    # create a max pooling layer
    pool = tf.nn.max_pool(x, ksize=[1, filter_height, filter_weight,1],
                  strides=[1, stride_x, stride_y, 1],
                  padding=padding, name=name)
    return pool

def lrn(x, radius, alpha, beta, name, bias=1.0):
    # create a local response normalization layer
    lrnl = tf.nn.local_response_normalization(x, depth_radius=radius,
                                             alpha=alpha, beta=beta,
                                             bias=bias, name=name)
    return lrnl

def dropout(x, keep_prob):
    # create a dropout layer
    return tf.nn.dropout(x, keep_prob)

#### 3.2 implementation of Alexnet architecture

In [30]:
def AlexNet(x, keep_prob, num_classes):
    # create the network graph
    # 1st layer:  Conv (w ReLu) -> Lrn -> Pool
    #conv1 = conv(x, [11,11], 96, [4,4], padding="VALID", name="conv1")
    conv1 = conv(x, [3,3], 96, [2,2], padding="VALID", name="conv1")
    norm1 = lrn(conv1, 2, 2e-5, 0.75, name="norm1")
    pool1 = max_pool(norm1, [3,3], [2,2], padding="VALID", name="pool1")
    
    # 2nd layer: Conv (w ReLu) -> Lrn -> Pool
    conv2 = conv(pool1, [5,5], 256, [1,1], name="conv2")
    norm2 = lrn(conv2, 2, 2e-5, 0.75, name="norm2")
    pool2 = max_pool(norm2, [3,3], [2,2], padding="VALID", name="pool12")
    
    # 3rd layer: Conv (w ReLu)
    conv3 = conv(pool2, [3,3], 384, [1,1], name="conv3")
    
    # 4rd layer: Conv (w ReLu)
    conv4 = conv(conv3, [3,3], 384, [1,1], name="conv4")
    
    # 5th Layer: Conv (w ReLu) -> Pool
    conv5 = conv(conv4, [3,3], 256, [1,1], name="conv5")
    pool5 = max_pool(conv5, [3,3], [2,2], padding="VALID", name="pool5")
    
    # 6th Layer: Flatten -> FC (w ReLu) -> Dropout
    #flattened = tf.reshape(pool5, [-1, 6x6x256])
    flattened = tf.reshape(pool5, [-1, 2x2x256])
    #fc6 = fc(flattened, 6x6x256, 4096, name="fc6")
    fc6 = fc(flattened, 2x2x256, 4096/8, name="fc6")
    dropout6 = dropout(fc6, keep_prob)
    
    # 7th Layer: FC (w ReLu) -> Dropout
    fc7 = fc(dropout6, 4096/8, 4096/8, name="fc7")
    dropout7 = dropout(fc7, keep_prob)
    
    # 8th Layer: FC (without ReLu activations) 
    fc8 = fc(dropout7, 4096/8, num_classes, relu=False, name="fc8")
    
    return fc8

#### 3.3 finetune_alexnet

In [36]:
tf.reset_default_graph()
keep_prob = 0.5
num_classes = 10
num_epochs = 10
learning_rate = 0.01
batch_size = 32

with tf.name_scope("input"):
    x = tf.placeholder(tf.float32,[batch_size, 28, 28, 1])
    y = tf.placeholder(tf.float32, [batch_size ,num_classes])
    
with tf.variable_scope("model_architecture"):
    fc8 = AlexNet(x, keep_prob, num_classes)
    
with tf.name_scope("loss"):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=fc8,labels=y)
    loss = tf.reduce_mean(cross_entropy)

tf.summary.scalar("loss",loss)
    
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(loss)

with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(fc8,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

tf.summary.scalar("accuracy",accuracy)
    
mergerd_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter(r"c:\users\bingo\desktop\log")

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    next_batch = get_data(batch_size, num_epochs)
    writer.add_graph(sess.graph)
    for epoch in range(num_epochs):
        img_batch, label_batch = sess.run(next_batch)
        img_batch = np.reshape(img_batch,[-1,28,28,1])
        label_batch = sess.run(tf.one_hot(label_batch, num_classes))
        
        _, summary = sess.run([train_op,mergerd_summary], feed_dict={x:img_batch, y:label_batch}) 
        writer.add_summary(summary,epoch)
    writer.close()